In [57]:
# %pip install numpy pandas duckdb

In [58]:
import numpy
import pandas as pd
import duckdb

## SQL Queries over Data Files With DuckDB

SQL, which stands for Structured Query Language, is a standard language used to communicate with databases. It is widely used for managing and manipulating data in databases, and is particularly good for tabular-formatted data. SQL allows you to perform tasks such as retrieving specific data from large databases, updating data, creating new tables or databases, and setting permissions on data access. It is a fundamental tool in many fields that require data management, including research, business, and software development.

[DuckDB](https://duckdb.org/) is a database management system that is designed to be easy to use and efficient. It is useful for anyone who needs to work with big datasets and wants to get results quickly, and [it has bindings in a wide variety of languages (including Python and a command-line interface)](https://duckdb.org/docs/api/overview).  Whether you are working on research data or any large dataset, DuckDB can be a helpful tool for managing and analyzing your data. Some other properties:

  - DuckDB is an **"embedded DBMS***, like SQLite, meaning that it it doesn't need a server process to run. 
  - DudckDB is a **"column-store"** DBMS, which makes it very fast at analytical processes like loading up a few large columns at a time.  The tradeoff is that it is slower at writing transactional data, so it's not meant to be used for writing large amounts real-time data at high speeds (for this, SQLite, MySQL, and PostGres would be popular alternatives). 
  - DuckDB is unique in that it can be used directly on common tabular file types, without necessarily building a special database file and defining all the tables first.  
  - DuckDB has integrations with Pandas, Polars, and PyArrow, and can both run queries on those Python variables without making any copies (high-speed!), and can convert its tables directly to those types, making it easy to integrate into most analysis pipelines.
  

In this notebook, we'll practice writing SQL queries on *JSON*, *CSV*, and *PARQUET* file formats.

## Loading Data from DataFrames with DuckDB: the SELECT * FROM Statement with Pandas

| Code | Description |
| :-- | :-- |
| `duckdb.sql('SELECT * FROM df')` | Load all the data from the `df` variable into DuckDB |
| `duckdb.sql('FROM df')` | A shortcut: The same as "SELECT * FROM df" |
| `duckdb.sql('SELECT colA, colB FROM df')` | Load only columns "colA" and "colA" from `df`. |
| `duckdb.sql('DESCRIBE SELECT * FROM df')` | Show information about each column in `df`. |
| `duckdb.sql('DESCRIBE FROM df')` | Show information about each column in `df`. |
| `duckdb.sql('SELECT ColA, ColB FROM df ORDER BY ColA')` | Order the rows by the values in ColA |
| `duckdb.sql('SELECT ColA as ColumnA, * FROM df ORDER BY ColA')` | Rename ColA to ColumnA and select the rest of the columns as well. |
| `duckdb.sql('FROM df LIMIT 3)` | Show only the first 3 rows (good for quickly previewing data) |
| `duckdb.sql().to_df()` | Convert the result back to a Pandas DataFrame. |




**Exercises**

Use the following Dataframes to answer the questions below:

In [17]:
mri_sessions = pd.DataFrame({
    "SessionID": [101, 102, 103, 104, 105],
    "PatientID": ["P001", "P002", "P001", "P003", "P002"],
    "Date": ["2024-02-01", "2024-02-03", "2024-02-05", "2024-02-07", "2024-02-09"],
    "Duration": [30, 45, 30, 60, 45]
})

ca2_sessions = pd.DataFrame({
    "ExperimentID": [301, 302, 303, 304, 305],
    "AnimalModel": ["Mouse", "Rat", "Zebrafish", "Mouse", "Rat"],
    "SessionDate": ["2024-04-01", "2024-04-05", "2024-04-09", "2024-04-13", "2024-04-17"],
    "DyeUsed": ["OGB-1", "Fluo-4", "GCaMP6", "OGB-1", "Fluo-4"],
    "ImagingTechnique": ["Two-photon", "Confocal", "Light-sheet", "Two-photon", "Confocal"],
    "NumCells": [200, 150, 300, 250, 180]
})

ca2_sessions

,ExperimentID,AnimalModel,SessionDate,DyeUsed,ImagingTechnique,NumCells
0,301,Mouse,2024-04-01,OGB-1,Two-photon,200
1,302,Rat,2024-04-05,Fluo-4,Confocal,150
2,303,Zebrafish,2024-04-09,GCaMP6,Light-sheet,300
3,304,Mouse,2024-04-13,OGB-1,Two-photon,250
4,305,Rat,2024-04-17,Fluo-4,Confocal,180


**Example**: Get all columns of the mri session dataframe:

In [18]:
sql("FROM mri_sessions")

┌───────────┬───────────┬────────────┬──────────┐
│ SessionID │ PatientID │    Date    │ Duration │
│   int64   │  varchar  │  varchar   │  int64   │
├───────────┼───────────┼────────────┼──────────┤
│       101 │ P001      │ 2024-02-01 │       30 │
│       102 │ P002      │ 2024-02-03 │       45 │
│       103 │ P001      │ 2024-02-05 │       30 │
│       104 │ P003      │ 2024-02-07 │       60 │
│       105 │ P002      │ 2024-02-09 │       45 │
└───────────┴───────────┴────────────┴──────────┘

Get all columns of the ca2 sessions:

Select only the Patient ID and Duration columns from the MRI data

Select only the Experiment ID and Animal Model columns from the MRI data

Reorder the CA2 data by the number of cells counted, and convert the result back to a Pandas DataFrame:

In the Ca2 data, rename the `SessionDate` column to be just "`Date`".

In the MRI data, load just the `SessionID` (renamed to be just `ID`) and (`PatientID` renamed to be `Patient`) columns. Sort the data by the patient ID.

## Loading Data with DuckDB: the SELECT * FROM Statement with Files

Now, let's build on these queries, using DuckDB's ability to work on 

| Code | Description |
| :-- | :-- |
| `duckdb.sql('SELECT * FROM "path/to/file.json"')` | [Load the JSON file into a table.](https://duckdb.org/docs/data/json/overview) |
| `duckdb.sql('SELECT * FROM "path/to/file.csv"')` | [Load the CSV file into a table.](https://duckdb.org/docs/data/csv/overview) |
| `duckdb.sql('SELECT * FROM "path/to/file.parquet"')` | [Load the PARQUET file into a table.](https://duckdb.org/docs/data/parquet/overview) |
| `duckdb.sql('SELECT * FROM "path/*/file.parquet"')` | Load **all** the "file.parquet" files into a table.|
| `duckdb.sql('SELECT * FROM "**/*.parquet"')` | Load **all** the parquet files that start with the letter "f" into a table, no matter what subfolder they are in. |
| `duckdb.sql('SELECT * FROM read_json("path/to/file.json", filename=true)')` | Load the "file.json" file into a table using the read_json() function directly. |




#### Download the Data

For the next sections, we'll be exploring some data from a Steinmetz et al NeuroPixel experiment, processed here into JSON files for our tabular analysis, along with some other familiar file types.  Please run the code below to download the data.  It will take 5-10 minutes to download.

In [ ]:
%pip install tqdm webdav4 requests fsspec

In [ ]:
from tqdm import tqdm
from pathlib import Path
from webdav4.fsspec import WebdavFileSystem

# https://uni-bonn.sciebo.de/s/oZql1bk0p1AvK0w
fs = WebdavFileSystem("https://uni-bonn.sciebo.de/public.php/webdav", auth=("oZql1bk0p1AvK0w", ""))
for name in tqdm(fs.ls("/", detail=False), desc="Downloading Data to data/stenmetz"):
    if not Path(f"data/steinmetz/{name}").exists():
        fs.download(name, f"data/steinmetz/{name}", recursive=True)

**Exercises**

In [1]:
from duckdb import sql

**Example**: Use duckdb to read the `session.json` file from the session recorded on 2017-10-11.  What metadata fields were recorded on that day?

In [3]:
sql('FROM "data/steinmetz/*_2017-10-11_*/session.json"')

┌──────────────┬─────────┬────────────┬──────────┬─────────┐
│ session_date │  mouse  │ stim_onset │ bin_size │   id    │
│     date     │ varchar │   double   │  double  │ varchar │
├──────────────┼─────────┼────────────┼──────────┼─────────┤
│ 2017-10-11   │ Theiler │        0.5 │     0.01 │ ab16    │
└──────────────┴─────────┴────────────┴──────────┴─────────┘

Use duckdb to read the `session.json` file from the session recorded on 2017-11-02.  What metadata fields were recorded on that day?

Use duckdb to read the 'trials.csv' file recorded on 2017-11-02.  What trial variables were recorded for that session?  (Note: if you cannot see all the columns, either add `"DESCRIBE"` to the front of the SQL statement, or you can convert the output to a Pandas DataFrame with `.to_df()`)

Use duckdb to read **all** of the `session.json` files in the dataset.

Use DuckDB to read all the `trials.csv` files in the dataset.

Use DuckDB to get all the cells that were recorded from in the dataset.

What variables were stored when recording lick behaviors (Tip: Use `DESCRIBE` at the front of the query)?

## Getting Unique Values with SELECT DISTINCT

Often, we want to summarize data by finding out what unique values there are in a dataset, in order to plan out an analysis.  In SQL, the [`SELECT DISTINCT`](https://duckdb.org/docs/sql/query_syntax/select) statement helps with this, and can even show distinct *combinations* of column values. 


| Code | Description |
| :-- | :-- |
| `SELECT * FROM my_table` |  Select all the columns. |
| `SELECT colA, colB FROM my_table` | Select just colA and colB |
| `SELECT DISTINCT colA FROM my_table` | Show just the unique values of colA |
| `SELECT DISTINCT colA, colB FROM my_table` | Show just the unique combinations of values between colA and colB |
| `SELECT DISTINCT colA, colB FROM my_table ORDER BY colA` | ...and order by colA |
| `SELECT DISTINCT colA, colB FROM my_table ORDER BY colA, colB` | ...and order by colA, then colB |
| `SELECT DISTINCT colA, colB FROM my_table ORDER BY *` | Order the data by whatever columns are selected, in the order they appear in the table. |

**Exercises**

**Example**: What mice were in the study?  Show just the unique values. 

In [41]:
sql(
"""
SELECT DISTINCT 
    mouse,
FROM 
    "data/steinmetz/*/session.json"
""")

┌───────────┐
│   mouse   │
│  varchar  │
├───────────┤
│ Lederberg │
│ Richards  │
│ Tatum     │
│ NULL      │
│ Moniz     │
│ Hench     │
│ Muller    │
│ Forssmann │
│ Theiler   │
│ Radnitz   │
├───────────┤
│  10 rows  │
└───────────┘

What unique ("distinct") contrast levels of the left stimulus were there in this experiment?  To make it easier to read, order the rows in the resulting table.

What unique ("distinct") combinations of contrast levels between the left and right stimulus were there in this experiment?  To make it easier to read, order the rows in the resulting table.

What were the different stimulus onset time settings used this experiment?

Which general areas of the  brain (let's use "brain_groups" here) were the cells in this study recorded from?

Which brain groups were associated with which brain areas?

## Filtering Data with WHERE

Let's Filter the data! The [WHERE](https://duckdb.org/docs/sql/query_syntax/where) clause in SQL helps to only load up the rows that meet a specified condition:

| Code | Description |
| :-- | :-- |
| `FROM my_table WHERE colA > 5` | Only take the rows where colA is greater than 5. |
| `FROM my_table WHERE colA = 'adult'` |  Only take the rows where colA is "adult". |
| `FROM my_table WHERE colA IS NOT NULL` | Only take the non-missing rows of colA |
| `FROM my_table WHERE colA LIKE 'C%'` | Only take the rows of ColA where the text starts with a "C". |

More filtering experessions can be found at these links (won't be used in these exercises; just for reference):
  - Logical Operators: https://duckdb.org/docs/sql/expressions/logical_operators
  - More Comparisons: https://duckdb.org/docs/sql/expressions/comparison_operators
  - More: https://duckdb.org/docs/sql/expressions/in
  - Text processing functions: https://duckdb.org/docs/sql/functions/char


**Exercises**

**Example**: Which brain areas of the brain group "thalamus" did we record cells from in this experiment?  (Tip: use single quotes `'` to reference text.)

In [59]:
sql(
"""
SELECT DISTINCT
    brain_groups,
    brain_area
FROM 
    'data/steinmetz/*/cells.parquet'
WHERE brain_groups = 'thalamus'
ORDER BY brain_groups
""")

┌──────────────┬────────────┐
│ brain_groups │ brain_area │
│   varchar    │  varchar   │
├──────────────┼────────────┤
│ thalamus     │ MG         │
│ thalamus     │ VAL        │
│ thalamus     │ RT         │
│ thalamus     │ PT         │
│ thalamus     │ SPF        │
│ thalamus     │ LD         │
│ thalamus     │ LP         │
│ thalamus     │ LH         │
│ thalamus     │ MD         │
│ thalamus     │ TH         │
│ thalamus     │ PO         │
│ thalamus     │ VPM        │
│ thalamus     │ CL         │
│ thalamus     │ LGd        │
│ thalamus     │ VPL        │
│ thalamus     │ POL        │
├──────────────┴────────────┤
│ 16 rows         2 columns │
└───────────────────────────┘

Which brain areas recorded in this experiment start with the capital letter `V`?

Which brain groups recorded in this experiment end with the word `"cortex"`?

Which sessions had cell recordings in the visual cortex?

Which sessions had recordings in the hippocampus?

Select all the "active" trials (when the mouse was expected to respond), where they actually responded with a wheel turn (response type was -1 or 1, not 0).